In [1]:
import idx2numpy
import seaborn as sns
X_train_3D = idx2numpy.convert_from_file('MNIST_dataset/train-images-idx3-ubyte')
X_train = X_train_3D.flatten().reshape(60000,784)

y_train = idx2numpy.convert_from_file('MNIST_dataset/train-labels-idx1-ubyte')
print(X_train.shape, y_train.shape)
X_test_3D = idx2numpy.convert_from_file('MNIST_dataset/t10k-images-idx3-ubyte')
X_test =  X_test_3D.flatten().reshape(10000,784)

y_test = idx2numpy.convert_from_file('MNIST_dataset/t10k-labels-idx1-ubyte')
print(X_test.shape, y_test.shape)

(60000, 784) (60000,)
(10000, 784) (10000,)


In [2]:
import numpy as np
from sklearn.utils import shuffle
X_shuffle,y_shuffle = shuffle(X_train,y_train)
X_train = X_shuffle[0:50000]
y_train = y_shuffle[0:50000]

from sklearn import tree
from sklearn.model_selection import cross_val_predict

#dt_clf = tree.DecisionTreeClassifier(max_depth=20, max_leaf_nodes=300)
dt_clf = tree.DecisionTreeClassifier(max_depth=8)

y_train_pred = cross_val_predict(dt_clf, X_train, y_train, cv=2)
print(dt_clf.fit(X_train, y_train))

print('Number of nodes in the decision tree {}.'.format(dt_clf.tree_.node_count))
print('Number of threshold in the decision tree {}.'.format(len(dt_clf.tree_.threshold)))
print('Number of leaves in the decision tree {}. It will be number of states in sequential FSM.'.format(dt_clf.tree_.n_leaves))

from sklearn.metrics import accuracy_score
print(accuracy_score(y_train, dt_clf.predict(X_train)), accuracy_score(y_test, dt_clf.predict(X_test)))

DecisionTreeClassifier(max_depth=8)
Number of nodes in the decision tree 509.
Number of threshold in the decision tree 509.
Number of leaves in the decision tree 255. It will be number of states in sequential FSM.
0.82916 0.8135


In [3]:
import pickle
with open('MNIST_dataset/MNIST_dataset.pkl', 'wb') as f:
    pickle.dump([X_train, y_train, X_test, y_test], f)

#### Run this cell twice.

In [5]:
%%capture cap --no-stderr
from sklearn.tree import _tree

def tree_to_code(tree, feature_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]
    
    def recurse(node, depth):
        indent = "  " * depth
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            print ("{}{}:if ({} <= {})".format(indent, node, name, int(round(threshold,3)))) 
            recurse(tree_.children_left[node], depth + 1)
            print ("{}{}:else ".format(indent, node, name, int(round(threshold,3))))
            recurse(tree_.children_right[node], depth + 1)
        else:
            print ("{} Label<={};".format(indent, np.argmax(tree_.value[node][0],axis=0)))

    recurse(0, 1)

cols = range(784)
features = ['pixels[{}]'.format(str(i)) for i in cols]
class_names = [str(i) for i in dt_clf.classes_]
tree_to_code(dt_clf, features)

with open('decision_rules_mnist.txt', 'w') as f:
    f.write(cap.stdout)

#### Run this cell twice.

In [7]:
%%capture cap --no-stderr

import pdb
def write_file(data_towrite):
    with open('verilog_file.v', 'a', encoding='utf-8') as file:
        file.writelines(data_towrite)
        file.close()
        
def write_line_file(data_towrite, line_num):
    with open('verilog_file.v', 'r', encoding='utf-8') as file:
        file_content = file.readlines()
        file.close()
    file_content[line_num] = file_content[line_num].replace('\n','') + ' ' + data_towrite + '\n'  
    with open('verilog_file.v', 'w', encoding='utf-8') as file:
        file.writelines(file_content)
        file.close()
        
def search_content_file(word):
    with open('verilog_file.v', 'r') as file:
        file_content = file.readlines()
        for line_num,line in enumerate(file_content):
            if word in line:
                return(line_num)
        return("content doesn't exists.")
    
def update_stateformat_file():
    with open('verilog_file.v', 'r', encoding='utf-8') as file:
        file_content = file.readlines()
        file.close()
    for line_num,line in enumerate(file_content):
        if 'if' in line:
            file_content[line_num] = '    case ' + file_content[line_num].strip() + ' \n'  
    with open('verilog_file.v', 'w', encoding='utf-8') as file:
        file.writelines(file_content)
        file.close()
        
def clear_file():
    with open('verilog_file.v', 'w', encoding='utf-8') as file:
        pass
        file.close()
        
def print_file_content():
    with open('verilog_file.v', 'r', encoding='utf-8') as file:
        file_content = file.read()
        file.close()
    print(file_content)


import itertools
with open('decision_rules_mnist.txt', 'r') as f:
    tree_verilog = f.read()
    f.close()

clear_file()
line_num = 0
else_state = 0
else_nextif_state = 0
curr_line,next_line = itertools.tee(tree_verilog.split('\n'))
next(next_line, None)
for i,j in list(zip(curr_line,next_line)):
    if 'Label' in i:
        pass
        #write_file(f'  begin {i.strip()} state<=0;ml_inference_completed<=1; end \n')
    elif 'else' in i and 'Label' in j:
        else_state = i.strip().split(':')[0]
        line_num = search_content_file(' '+str(else_state)+':if' )
        spaces = 12-len(str(line_num))
        spaces = ' '*spaces
        write_line_file(f'\n{spaces}else:  {j.strip().replace("<","")} state=0;ml_inference_completed=1; clock_counter = clock_counter+1;', line_num)
    elif 'else' in i and 'if' in j:
        else_state = i.strip().split(':')[0]
        else_nextif_state = j.strip().split(':')[0]
        line_num = search_content_file(' '+str(else_state)+':if' )
        spaces = 12-len(str(line_num))
        spaces = ' '*spaces
        write_line_file(f' \n{spaces}else: state={else_nextif_state}; clock_counter = clock_counter+1;', line_num)
    elif 'if' in i and 'Label' in j:
        write_file(f' {i.strip()}:  {j.strip().replace("<","")} state=0;ml_inference_completed<=1; clock_counter = clock_counter+1; ')
    elif 'if' in i and 'if' in j:
        write_file(f' {i.strip()}: clock_counter = clock_counter+1; state={j.strip().split(":")[0]};  \n ')  
update_stateformat_file()
print('def inference_rate(pixels):')
print(' clock_counter=0; state=0; Label=10; ml_inference_completed=0; #assigning dummy default label to 10')
print(' while(Label == 10):')
print('  match state:')
print_file_content()
print(' return (Label,clock_counter)')

with open('mnist_dt_case_fsm.py', 'w') as f:
    f.write(cap.stdout)

In [8]:
# Aligning the spaces
with open('mnist_dt_case_fsm.py', 'r') as f:
    py_fsm = f.readlines()
    f.close()
    
curr_line,next_line = itertools.tee(py_fsm)
next(next_line, None)
for i,j in list(zip(curr_line,next_line)):
    if 'if' in i and 'else' in j:
        if len(i.split('if')[0]) != len(j.split('else')[0]):
            py_fsm[py_fsm.index(j)] = ' '* (len(i.split('if')[0]) - len(j.split('else')[0])) + py_fsm[py_fsm.index(j)]
            

for i in py_fsm:
    if 'case' in i:
        py_fsm[py_fsm.index(i)] = i.split(':if')[0]+':\n'+ ' '*(len(i.split(':if')[0])+1) + 'if' + i.split(':if')[1]
        
with open('mnist_dt_case_fsm.py', 'w') as f:
    f.writelines(py_fsm)
    f.close()        

def inference_rate(pixels):
 clock_counter=0; state=0; Label=10; ml_inference_completed=0; #assigning dummy default label to 10
 while(Label == 10):
  match state:
    case 0:
           if (pixels[350] <= 121): clock_counter = clock_counter+1; state=1; 
           else: state=254; clock_counter = clock_counter+1;
    case 1:
           if (pixels[435] <= 0): clock_counter = clock_counter+1; state=2; 
           else: state=127; clock_counter = clock_counter+1;
    case 2:
           if (pixels[597] <= 1): clock_counter = clock_counter+1; state=3; 
           else: state=66; clock_counter = clock_counter+1;
    case 3:
           if (pixels[458] <= 1): clock_counter = clock_counter+1; state=4; 
           else: state=35; clock_counter = clock_counter+1;
    case 4:
           if (pixels[184] <= 2): clock_counter = clock_counter+1; state=5; 
           else: state=20; clock_counter = clock_counter+1;
    case 5:
           if (pixels[404] <= 1): clock_counter = clock_counter+1; state=6;

In [9]:
#Adding default state 
with open('mnist_dt_case_fsm.py', 'r') as f:
    py_fsm = f.readlines()
    f.close()
py_fsm[-2] = ' '*4+'case _:\n'+ ' '*len(' '*4+'case _:') + 'Label=10; state=0;ml_inference_completed=1; clock_counter = clock_counter+1;\n'

with open('mnist_dt_case_fsm.py', 'w') as f:
    f.writelines(py_fsm)
    f.close()        
    
with open('mnist_dt_case_fsm.py', 'r') as f:
    py_fsm_ap = f.read()
    f.close()
    
print(py_fsm_ap)

def inference_rate(pixels):
 clock_counter=0; state=0; Label=10; ml_inference_completed=0; #assigning dummy default label to 10
 while(Label == 10):
  match state:
    case 0:
           if (pixels[350] <= 121): clock_counter = clock_counter+1; state=1; 
           else: state=254; clock_counter = clock_counter+1;
    case 1:
           if (pixels[435] <= 0): clock_counter = clock_counter+1; state=2; 
           else: state=127; clock_counter = clock_counter+1;
    case 2:
           if (pixels[597] <= 1): clock_counter = clock_counter+1; state=3; 
           else: state=66; clock_counter = clock_counter+1;
    case 3:
           if (pixels[458] <= 1): clock_counter = clock_counter+1; state=4; 
           else: state=35; clock_counter = clock_counter+1;
    case 4:
           if (pixels[184] <= 2): clock_counter = clock_counter+1; state=5; 
           else: state=20; clock_counter = clock_counter+1;
    case 5:
           if (pixels[404] <= 1): clock_counter = clock_counter+1; state=6;

## FSM State Permutation Obfuscation

#### Extracting the state labels and transition labels.

In [10]:
with open('mnist_dt_case_fsm.py', 'r') as f:
    py_fsm = f.readlines()
    f.close()
    
state_labels = []
transition_labels = []
for i in py_fsm:
    if 'case' in i and '_' not in i:
        state_labels.append(int(i.split(':')[0].split(' ')[-1]))
    if 'state=' in i:
        transition_labels.append(int(i.split('state=')[-1].split(';')[0]))

transition_labels = list(set(transition_labels))
print(state_labels == transition_labels)

print(f'Number of state/transition labels = {len(state_labels)}.')
print(state_labels)

True
Number of state/transition labels = 254.
[0, 1, 2, 3, 4, 5, 6, 7, 10, 13, 14, 17, 20, 21, 22, 25, 28, 29, 32, 35, 36, 37, 38, 41, 44, 45, 48, 51, 52, 53, 56, 59, 60, 63, 66, 67, 68, 69, 70, 74, 75, 78, 81, 82, 83, 86, 89, 90, 93, 96, 97, 98, 99, 102, 105, 106, 109, 112, 113, 114, 117, 120, 121, 124, 127, 128, 129, 130, 131, 132, 135, 138, 139, 142, 145, 146, 147, 150, 153, 154, 157, 160, 161, 162, 163, 166, 169, 170, 173, 176, 177, 178, 181, 184, 185, 188, 191, 192, 193, 194, 195, 198, 201, 202, 205, 208, 209, 210, 213, 216, 217, 220, 223, 224, 225, 226, 229, 232, 233, 236, 239, 240, 241, 244, 247, 248, 251, 254, 255, 256, 257, 258, 259, 260, 263, 266, 267, 270, 273, 274, 275, 278, 281, 282, 285, 288, 289, 290, 291, 294, 297, 298, 301, 304, 305, 306, 309, 312, 313, 316, 319, 320, 321, 322, 323, 326, 329, 330, 333, 336, 337, 338, 341, 344, 345, 348, 351, 352, 353, 354, 357, 360, 361, 364, 367, 368, 369, 372, 375, 376, 379, 382, 383, 384, 385, 386, 387, 390, 393, 394, 397, 400, 401,

#### Designing a correct key for state label.

In [23]:
import math
max_val_statelabel = max(state_labels)
number_of_bits = math.floor(math.log2(abs(max(state_labels)))) + 1
print(f'Maximum vlaue of state label = {max_val_statelabel}. ')
print(f'Minimum number of bits to represent it = {number_of_bits}.')

def convert_to_binary_with_padding(numbers):
    max_bits = len(bin(max(numbers))) - 2  # Determine the maximum number of bits needed
    binary_numbers = []
    for num in numbers:
        binary = bin(num)[2:]  # Convert number to binary, remove '0b' prefix
        padded_binary = binary.zfill(max_bits)  # Perform zero padding
        binary_numbers.append(padded_binary)
    return binary_numbers

binary_state_labels = convert_to_binary_with_padding(state_labels)
bin_correct_secret_key = ''.join(binary_state_labels)
print(f'Length of bin_correct_secret_key before 8-bit word division = {len(bin_correct_secret_key)}.')
last_chunk_size = len(bin_correct_secret_key) % 8    #Checking the last chunk whether it is 8-bit or not
if last_chunk_size !=0:
    bin_correct_secret_key = bin_correct_secret_key[0:-last_chunk_size] + bin_correct_secret_key[-last_chunk_size:].zfill(8)
print(f'Length of bin_correct_secret_key after 8-bit word division = {len(bin_correct_secret_key)}.')
print(f'correct_secret_key in binary = {bin_correct_secret_key}')

def binary_stream_to_bytes(binary_stream):
    # Split the binary stream into 8-bit chunks
    chunks = [binary_stream[i:i+8] for i in range(0, len(binary_stream), 8)]
    # Convert each chunk to a decimal integer
    bytes_list = [int(chunk, 2) for chunk in chunks]
    return bytes_list

decimal_correct_secret_key = binary_stream_to_bytes(bin_correct_secret_key)
print(f'correct_secret_key in decimal = {decimal_correct_secret_key}')
print(f'Length of correct_secret_key in decimal = {len(decimal_correct_secret_key)}. This matches |bin_correct_secret_key|/8 = {len(bin_correct_secret_key)/8}.')

Maximum vlaue of state label = 506. 
Minimum number of bits to represent it = 9.
Length of bin_correct_secret_key before 8-bit word division = 2286.
Length of bin_correct_secret_key after 8-bit word division = 2288.
correct_secret_key in binary = 000000000000000001000000010000000011000000100000000101000000110000000111000001010000001101000001110000010001000010100000010101000010110000011001000011100000011101000100000000100011000100100000100101000100110000101001000101100000101101000110000000110011000110100000110101000111000000111011000111100000111111001000010001000011001000100001000101001000110001001010001001011001001110001010001001010010001010011001010110001011001001011010001011101001100000001100001001100010001100011001100110001101001001101010001101101001110000001110001001110010001110101001111000001111001001111100001111111010000000010000001010000010010000011010000100010000111010001010010001011010001110010010001010010010010010011010010110010011001010011010010011101010100000010100001010100

#### Creating the state_label and transition_label encoding for each case statement in the mnist_dt_case_fsm.py. Saving the script in locked_mnist_dt_case_fsm.py

In [12]:
import pdb
with open('mnist_dt_case_fsm.py', 'r') as f:
    py_fsm = f.readlines()
    f.close()
    
#In Python >=3.10, the match statement doesn't support using variables or expressions as case labels directly. 
#However, you can achieve the desired functionality by using the case statement with conditions.

encoder_counter = 0
encoded_label_dict = {}  #read as : value = true case state label, key = encoded label sliced from the secret key
for i in py_fsm:
    if 'case' in i and '_' not in i:
        py_fsm[py_fsm.index(i)] = i.replace(i.split(':')[0].split(' ')[-1], f'x if x == encoded_state_label[{encoder_counter}]')
        encoded_label_dict[encoder_counter] = int(i.split(':')[0].split(' ')[-1])
        encoder_counter = encoder_counter+1

import random
def shuffle_dict(dictionary):
    keys = list(dictionary.keys())
    values = list(dictionary.values())
    random.shuffle(values)
    shuffled_dict = dict(zip(keys, values))
    return shuffled_dict

def find_key(dictionary, value):
    for key, val in dictionary.items():
        if val == value:
            return key
    return None
        
transition_label_dict = shuffle_dict(encoded_label_dict) #read as : vaue = true transition state label, key = encoded state label sliced from the secret key


#Need to apply regular expression because there are multiple integers in line 
for i in py_fsm:
    if ('state=' in i) and ('if' in i or 'else' in i):
        next_state = i.split('state=')[-1].split(';')[0]
        true_trans_state = find_key(transition_label_dict, int(next_state) )
        py_fsm[py_fsm.index(i)] = i.replace( 'state=' + next_state, f"state=encoded_transition_label[{true_trans_state}]")

py_fsm[0] =  py_fsm[0].replace('pixels', 'pixels, encoded_state_label, encoded_transition_label')
py_fsm[0] =  py_fsm[0].replace('inference', 'Lockedinference')

with open('locked_mnist_dt_case_fsm.py', 'w') as f:
    f.writelines(py_fsm)
    f.close()        
    

print(encoded_label_dict, '\n-----\n', transition_label_dict)

{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 10, 9: 13, 10: 14, 11: 17, 12: 20, 13: 21, 14: 22, 15: 25, 16: 28, 17: 29, 18: 32, 19: 35, 20: 36, 21: 37, 22: 38, 23: 41, 24: 44, 25: 45, 26: 48, 27: 51, 28: 52, 29: 53, 30: 56, 31: 59, 32: 60, 33: 63, 34: 66, 35: 67, 36: 68, 37: 69, 38: 70, 39: 74, 40: 75, 41: 78, 42: 81, 43: 82, 44: 83, 45: 86, 46: 89, 47: 90, 48: 93, 49: 96, 50: 97, 51: 98, 52: 99, 53: 102, 54: 105, 55: 106, 56: 109, 57: 112, 58: 113, 59: 114, 60: 117, 61: 120, 62: 121, 63: 124, 64: 127, 65: 128, 66: 129, 67: 130, 68: 131, 69: 132, 70: 135, 71: 138, 72: 139, 73: 142, 74: 145, 75: 146, 76: 147, 77: 150, 78: 153, 79: 154, 80: 157, 81: 160, 82: 161, 83: 162, 84: 163, 85: 166, 86: 169, 87: 170, 88: 173, 89: 176, 90: 177, 91: 178, 92: 181, 93: 184, 94: 185, 95: 188, 96: 191, 97: 192, 98: 193, 99: 194, 100: 195, 101: 198, 102: 201, 103: 202, 104: 205, 105: 208, 106: 209, 107: 210, 108: 213, 109: 216, 110: 217, 111: 220, 112: 223, 113: 224, 114: 225, 115: 226, 116: 229, 

#### Designing a correct key for transition label.

In [26]:
binary_transition_labels = convert_to_binary_with_padding(transition_label_dict.values())
transition_bin_correct_secret_key = ''.join(binary_transition_labels)
print(f'Length of transition_last_chunk_size before 8-bit word division = {len(transition_bin_correct_secret_key)}.')
transition_last_chunk_size = len(transition_bin_correct_secret_key) % 8    #Checking the last chunk whether it is 8-bit or not
if transition_last_chunk_size !=0:
    transition_bin_correct_secret_key = transition_bin_correct_secret_key[0:-transition_last_chunk_size] + transition_bin_correct_secret_key[-transition_last_chunk_size:].zfill(8)
print(f'Length of transition_bin_correct_secret_key after 8-bit word division = {len(transition_bin_correct_secret_key)}.')
print(f'transition_bin_correct_secret_key in binary = {transition_bin_correct_secret_key}')

transition_decimal_correct_secret_key = binary_stream_to_bytes(transition_bin_correct_secret_key)
print(f'transition_correct_secret_key in decimal = {transition_decimal_correct_secret_key}')
print(f'Length of transition_correct_secret_key in decimal = {len(transition_decimal_correct_secret_key)}. This matches |transition_bin_correct_secret_key|/8 = {len(transition_bin_correct_secret_key)/8}.')

Length of transition_last_chunk_size before 8-bit word division = 2286.
Length of transition_bin_correct_secret_key after 8-bit word division = 2288.
transition_bin_correct_secret_key in binary = 001111000111110000110000000101100000001111001110100010101110001010110000111011110011110000010110001100001010000111000110000110100010001110011001100101001110100000111000000110001101000000110000000011011111110100010110010010010000111100010001110000001010101011001000000001111001001010110101101111000010000001111011011110110000111101111101010010111111010001111111100100010011111011101001001100101010001001010101011000001011001110010010010010011100011010111010100100011001001101001001000011010000100100100110001100010100000000110010101011010101011100000011101001100101101011011000110011000011100101110110100110001001100000111110001010100111100111010001000010001111100100100011101000100101100111111111110011001100001101000110111001100010100000101001101101010101001110101011010001000101001101111110011000001001

#### Testing the locked model with correct encoding_label and transition_label.

In [13]:
from locked_mnist_dt_case_fsm import Lockedinference_rate

In [14]:
import pickle
with open('MNIST_dataset/MNIST_dataset.pkl', 'rb') as f:
    X_train, y_train, X_test, y_test = pickle.load(f)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(50000, 784) (50000,) (10000, 784) (10000,)


In [15]:
clock_counter = 0
Y_pred = []
for i,sample in enumerate(X_test):
    inf_label, clock = Lockedinference_rate(sample, encoded_label_dict, transition_label_dict)
    Y_pred.append(inf_label)
    clock_counter = clock_counter + clock

In [16]:
print(f'Total clock used for inferencing 10000 samples = {clock_counter}')
#Verify that the accuracy measured after state-machine inference is same as given by sklearn model.
from sklearn.metrics import accuracy_score
print(f'Accuracy using state machine = {accuracy_score(y_test, Y_pred)}')

Total clock used for inferencing 10000 samples = 79998
Accuracy using state machine = 0.8135


### Testing the model with correct key where encoding_labels and transition_labels are extracted from the key

#### Extracting the state_labels.

In [24]:
def convert_to_fixed_bit_binary(numbers, num_bits):
    binary_list = []
    for num in numbers:
        # Convert number to binary and remove the '0b' prefix
        binary = bin(num)[2:]
        # Pad zeros to the left to ensure fixed number of bits
        padded_binary = binary.zfill(num_bits)
        binary_list.append(padded_binary)
    return binary_list

Extracted_bin_correct_secret_key=''.join(convert_to_fixed_bit_binary(decimal_correct_secret_key, 8))
print(f"verification in binary={Extracted_bin_correct_secret_key == bin_correct_secret_key}")

#Removing the zero-padded if perfomed from the last 8-bit chunk
if last_chunk_size != 0:
    Extracted_bin_correct_secret_key= Extracted_bin_correct_secret_key[:-8] + Extracted_bin_correct_secret_key[-8:][-last_chunk_size:]

def binary_stream_to_numbers(stream, n):
    numbers = []
    for i in range(0, len(stream), n):
        word = stream[i:i+n]
        numbers.append(int(word, 2))
    return numbers

max_number_of_bits = math.floor(math.log2(abs(max(state_labels)))) + 1
Extracted_state_labels = binary_stream_to_numbers(''.join(Extracted_bin_correct_secret_key), max_number_of_bits)
print(f"verification in decimal for state labels={Extracted_state_labels == state_labels}.")

verification in binary=True
verification in decimal for state labels=True.


In [25]:
Extracted_encoded_label_dict = {}
for i,j in enumerate(Extracted_state_labels):
    Extracted_encoded_label_dict[i] = j
print(f'Verification Extracted_encoded_label_dict = {Extracted_encoded_label_dict == encoded_label_dict}.')

Verification Extracted_encoded_label_dict = True.


#### Extracting transition labels.

In [27]:
Extracted_transition_bin_correct_secret_key=''.join(convert_to_fixed_bit_binary(transition_decimal_correct_secret_key, 8))
print(f"verification in binary={Extracted_transition_bin_correct_secret_key == transition_bin_correct_secret_key}")

#Removing the zero-padded if perfomed from the last 8-bit chunk
if transition_last_chunk_size != 0:
    Extracted_transition_bin_correct_secret_key= Extracted_transition_bin_correct_secret_key[:-8] + Extracted_transition_bin_correct_secret_key[-8:][-transition_last_chunk_size:]

max_number_of_bits = math.floor(math.log2(abs(max(transition_label_dict.values())))) + 1
Extracted_transition_labels = binary_stream_to_numbers(''.join(Extracted_transition_bin_correct_secret_key), max_number_of_bits)
print(f"verification in decimal for transition labels={Extracted_transition_labels == list(transition_label_dict.values())}.")

verification in binary=True
verification in decimal for transition labels=True.


In [28]:
Extracted_transition_label_dict = {}
for i,j in enumerate(Extracted_transition_labels):
    Extracted_transition_label_dict[i] = j
print(f'Verification Extracted_transition_label_dict = {Extracted_transition_label_dict == transition_label_dict}.')

Verification Extracted_transition_label_dict = True.


#### Testing

In [30]:
from locked_mnist_dt_case_fsm import Lockedinference_rate
import pickle
with open('MNIST_dataset/MNIST_dataset.pkl', 'rb') as f:
    X_train, y_train, X_test, y_test = pickle.load(f)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

clock_counter = 0
Y_pred = []
for i,sample in enumerate(X_test):
    inf_label, clock = Lockedinference_rate(sample, Extracted_encoded_label_dict, Extracted_transition_label_dict)
    Y_pred.append(inf_label)
    clock_counter = clock_counter + clock
    
print(f'Total clock used for inferencing 10000 samples = {clock_counter}')
#Verify that the accuracy measured after state-machine inference is same as given by sklearn model.
from sklearn.metrics import accuracy_score
print(f'Accuracy using state machine = {accuracy_score(y_test, Y_pred)}')

(50000, 784) (50000,) (10000, 784) (10000,)
Total clock used for inferencing 10000 samples = 79998
Accuracy using state machine = 0.8135


## The following cells work with correct unlocked model.

#### Run the next experiments in Python version > 3.10 because 'case' statement is only avaiable after this python version. 

#### Next, called the function "inference_rate" from python file "mnist_dt_case_fsm.py".

#### Remember that the while loop is added in the windows source code to mimic the clock and continuously run the state machine. 

In [17]:
from mnist_dt_case_fsm import inference_rate

In [18]:
import pickle
with open('MNIST_dataset/MNIST_dataset.pkl', 'rb') as f:
    X_train, y_train, X_test, y_test = pickle.load(f)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(50000, 784) (50000,) (10000, 784) (10000,)


In [12]:
clock_counter = 0
Y_pred = []
for i,sample in enumerate(X_test):
    inf_label, clock = inference_rate(sample)
    Y_pred.append(inf_label)
    clock_counter = clock_counter + clock

In [19]:
print(f'Total clock used for inferencing 10000 samples = {clock_counter}')

Total clock used for inferencing 10000 samples = 79998


In [20]:
#Verify that the accuracy measured after state-machine inference is same as given by sklearn model.
from sklearn.metrics import accuracy_score
print(f'Accuracy using state machine = {accuracy_score(y_test, Y_pred)}')

Accuracy using state machine = 0.8135


In [21]:
#Sequential circuit using finite state machine
#Average clock used per sample
Average_clock_per_sample = clock_counter/len(X_test)

print(Average_clock_per_sample)

7.9998


In [22]:
clock_period = 1/12000000 # 12 MHz
time_per_sample = Average_clock_per_sample *  clock_period * 1000000
print('Inference time used per sample when operate at 12MHz = ',time_per_sample, 'micro sec.' ) 
print('Inference per sec = ', round(1000000/time_per_sample))

Inference time used per sample when operate at 12MHz =  0.66665 micro sec.
Inference per sec =  1500038
